# Запустить интерактивную сессию ZooKeeper CLI 

In [4]:
import logging
logging.basicConfig()

In [5]:
from kazoo.client import KazooClient

In [ ]:
zk = KazooClient()
zk.start()

mapR_1  | 2022-12-27 11:05:08,423 [myid:] - INFO  [SyncThread:0:ZooKeeperServer@694] - Established session 0x100006195eb0000 with negotiated timeout 10000 for client /172.23.0.1:50366

## Освоить основные команды команды

In [5]:
# Same as `ls /`
zk.get_children("/")

['coordinator', 'mynode', 'zookeeper', 'table', 'myconfig']

In [6]:
zk.get_children("/zookeeper"), zk.get_children("/zookeeper/quota")

(['quota'], [])

In [7]:
zk.delete('/mynode', recursive=True)

In [8]:
zk.create(path="/mynode", value=b"first_version")

'/mynode'

In [9]:
zk.get_children("/")

['coordinator', 'mynode', 'zookeeper', 'table', 'myconfig']

In [10]:
zk.get('/mynode')  # Returns tuple (Value, Metadata), same as get /mynode && stat /mynode

(b'first_version',
 ZnodeStat(czxid=874, mzxid=874, ctime=1642683616521, mtime=1642683616521, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=874))

In [11]:
zk.set('/mynode', value=b"second_version")

ZnodeStat(czxid=874, mzxid=875, ctime=1642683616521, mtime=1642683616656, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=14, numChildren=0, pzxid=874)

In [12]:
zk.get('/mynode') 

(b'second_version',
 ZnodeStat(czxid=874, mzxid=875, ctime=1642683616521, mtime=1642683616656, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=14, numChildren=0, pzxid=874))

## Создание пары зависмых узлов
Аналогично `create -s /mynode/child 'data'`

In [13]:
zk.create('/mynode/child', value=b'im_sequential', sequence=True)
zk.create('/mynode/child', value=b'im_sequential_too', sequence=True)

'/mynode/child0000000001'

In [14]:
zk.get_children('/mynode')

['child0000000000', 'child0000000001']

## Одновременные соединения и эфемерные узлы

In [15]:
zk1, zk2 = KazooClient(), KazooClient()
zk1.start()
zk2.start()

In [16]:
zk.delete('/mygroup', recursive=True)

True

In [17]:
zk.create('/mygroup', b'top_node')

'/mygroup'

In [18]:
zk1.create('/mygroup/grue', b'im_grue', ephemeral=True)
zk2.create('/mygroup/bleen', b'im_bleen', ephemeral=True)

'/mygroup/bleen'

In [19]:
zk.get_children('/mygroup')

['grue', 'bleen']

In [20]:
zk1.get('/mygroup/bleen')

(b'im_bleen',
 ZnodeStat(czxid=882, mzxid=882, ctime=1642683618005, mtime=1642683618005, version=0, cversion=0, aversion=0, ephemeralOwner=72059201354334288, dataLength=8, numChildren=0, pzxid=882))

In [21]:
zk.get_children('/mygroup')

['grue', 'bleen']

In [22]:
zk2.get('/mygroup/bleen')

(b'im_bleen',
 ZnodeStat(czxid=882, mzxid=882, ctime=1642683618005, mtime=1642683618005, version=0, cversion=0, aversion=0, ephemeralOwner=72059201354334288, dataLength=8, numChildren=0, pzxid=882))

In [23]:
zk1.stop(), zk2.stop()

(None, None)

In [24]:
zk.delete('/mygroup', recursive=True)

## Конфигурация приложения

In [25]:
zk.delete('/myconfig', recursive=True)

In [26]:
zk.create('/myconfig', b'sheep_count=1')

'/myconfig'

In [27]:
zk1 = KazooClient()
zk1.start()

In [28]:
def watch_node(state):
    print(state)

In [29]:
zk1.get('/myconfig', watch=watch_node)

(b'sheep_count=1',
 ZnodeStat(czxid=887, mzxid=887, ctime=1642683618342, mtime=1642683618342, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=887))

Визуальное представление примера реакции на изменение узнал клиента zk1

In [30]:
zk.set('/myconfig', b'sheep_count=2')

WatchedEvent(type='CHANGED', state='CONNECTED', path='/myconfig')

ZnodeStat(czxid=887, mzxid=889, ctime=1642683618342, mtime=1642683618487, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=887)

In [31]:
zk.set('/myconfig', b'sheep_count=3')  # Trigger (watcher) is resetted 

ZnodeStat(czxid=887, mzxid=890, ctime=1642683618342, mtime=1642683618548, version=2, cversion=0, aversion=0, ephemeralOwner=0, dataLength=13, numChildren=0, pzxid=887)